<a href="https://colab.research.google.com/github/gourab-sinha/Machine_Learning/blob/master/Decision%20Tree/Project_Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn import datasets
import pandas as pd
import numpy as np
import math

In [0]:
class DecisionTree:
  def __information_req(self,X,Y):
    isCategorical = True
    if(X.dtype.name=='int64' or X.dtype.name=='float64'):
      isCategorical = False
    
    if isCategorical:
      categories = set(X)
      info_req = 0
      overall_total = len(Y)
      for category in categories:
        indices = X[X[0]==category].index
        Y_selected = Y[indices]
        types = dict(Y_selected.value_counts().items())
        total = len(Y_selected)
        node_entropy = 0
        for cat,val in types.items():
          node_entropy+= (val/total)*math.log2(val/total)
        
        node_entropy = (-1)*node_entropy

        info_req+= (total/overall_total)*node_entropy
      
      return info_req


      


  def __gain(self,X,Y,feature):
    types = dict(Y.value_counts().items())
    total = len(Y)
    node_entropy = 0
    for cat,val in types.items():
      node_entropy += (val/total)*math.log2(val/total)
    
    node_entropy = (-1)*node_entropy

    info_req = self.__information_req(X[feature],Y)

    

  def decision_tree(self,X,Y, features,level):  
    types = dict(Y.value_counts().items())
    if len(features)==0 or len(types)==1:
      print("Level ",level)
      for cat,val in types.items():
        print("Count of ",cat,val)
      print("Current Entropy is ", 0)
      print("Reached leaf Node")
    
    selected_feature = ""
    max_gain = -1
    for feature in features:
      current_gain = self.__gain(X,Y,feature)
      if max_gain<current_gain:
        selected_feature = feature
        max_gain = current_gain
    
    # recursive call





# Level  0
# Count of  0(False)  =  1
# Count of  1(True)  =  3
# Current Entropy  is =  0.811278124459
# Splitting on feature  X1  with gain ratio  0.311278124459



In [44]:
# importing pandas as pd 
import pandas as pd 

# Creating the index 
idx = pd.DataFrame([4.5, 3.01, 2.0, 1.0], columns =['Type'])

# Print the Index 
idx

,Type
0,4.50
1,3.01
2,2.00
3,1.00


In [45]:
idx['Type'].dtype.name

'float64'